In [2]:
# !pip install transformers peft bitsandbytes accelerate trl pydantic-settings scipy codebleu

In [3]:
# DEFINE QUANTIZATION HERE. Choose from ("none" | "8bit" | "4bit")
QUANTIZATION = "4bit"#DO NOT CHANGE
DATASET = "codeparrot/xlcost-text-to-code" #DO NOT CHANGE
TESTSIZE = 200

In [4]:
#CHANGE THE FOLLOWING
EXPERT = "Java" #python, cpp, java, javascript, etc.
MODELTYPE = "BASE" #BASE or FINETUNED or MOE
model_id = "mistralai/Mistral-7B-v0.1"

In [5]:
from huggingface_hub import login
login(token="")

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/ec2-user/.cache/huggingface/token
Login successful


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
import torch
import transformers
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, PeftModel

In [7]:
from datasets import load_dataset
PARTITION = "Java-program-level" #java-program-level etc.
data = load_dataset(DATASET, PARTITION)

Generating validation split: 100%|██████████| 494/494 [00:00<00:00, 160993.49 examples/s]


In [8]:
next(iter(data["test"]))

{'text': 'Minimum sum possible by removing all occurrences of any array element | Java program for the above approach ; Function to find minimum sum after deletion ; Stores frequency of array elements ; Traverse the array ; Calculate sum ; Update frequency of the current element ; Stores the minimum sum required ; Traverse map ; Find the minimum sum obtained ; Return minimum sum ; Driver code ; Input array ; Size of array',
 'code': 'import java . util . * ; class GFG { static int minSum ( int A [ ] , int N ) { HashMap < Integer , Integer > mp = new HashMap < Integer , Integer > ( ) ; int sum = 0 ; for ( int i = 0 ; i < N ; i ++ ) { sum += A [ i ] ; if ( mp . containsKey ( A [ i ] ) ) { mp . put ( A [ i ] , mp . get ( A [ i ] ) + 1 ) ; } else { mp . put ( A [ i ] , 1 ) ; } } int minSum = Integer . MAX_VALUE ; for ( Map . Entry < Integer , Integer > it : mp . entrySet ( ) ) { minSum = Math . min ( minSum , sum - ( it . getKey ( ) * it . getValue ( ) ) ) ; } return minSum ; } public stat

In [9]:
train_dataset = data["train"]
eval_dataset = data["validation"]
test_dataset = data["test"]

In [10]:
print(train_dataset)
print(eval_dataset)
print(test_dataset)

Dataset({
    features: ['text', 'code'],
    num_rows: 9623
})
Dataset({
    features: ['text', 'code'],
    num_rows: 494
})
Dataset({
    features: ['text', 'code'],
    num_rows: 911
})


In [11]:
# Pre-define quantization configs

################## 4bit ##################
bb_config_4b_eval = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
##########################################

################## 8bit ##################
bb_config_8b_eval = BitsAndBytesConfig(
    load_in_8bit=True,
)
##########################################

def quantization_config_eval(quantization):
    if quantization == "8bit":
        return bb_config_8b_eval
    else:
        return bb_config_4b_eval

In [12]:
import torch
import transformers
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, PeftModel

if QUANTIZATION == "none":
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        trust_remote_code=True,
        use_auth_token=True
    ).to("cuda")
else:
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=quantization_config_eval(QUANTIZATION),
        device_map="auto",
        trust_remote_code=True,
        use_auth_token=True
    )

tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    model_max_length=512,
    padding_side="left",
    add_eos_token=True)

tokenizer.pad_token = tokenizer.eos_token

/opt/conda/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:466: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [02:05<00:00, 62.80s/it]


In [13]:
eval_prompt = """Write a function to output the prime factorization of 2023 in Java, Javascript, C, and C++"""

In [14]:
# eval_prompt = f"""<s>[INST] Write a {EXPERT} program to complete the following. Minimum sum possible by removing all occurrences of any array element | Function to find minimum sum after deletion ; Stores frequency of array elements ; Traverse the array ; Calculate sum ; Update frequency of the current element ; Stores the minimum sum required ; Traverse map ; Find the minimum sum obtained ; Return minimum sum ; Input array ; Size of array [/INST] \\n code:</s>"""

In [15]:
# eval_prompt = f"""<s>
# For a given integer n, print out all its prime factors one on each line. 
# n = 30
# [INST]
# """
model.eval()
with torch.no_grad():
  input_ids = tokenizer(eval_prompt, return_tensors="pt", truncation=True).input_ids.cuda()
  outputs = model.generate(input_ids=input_ids, max_new_tokens=256, do_sample=True, top_p=0.9,temperature=0.5)

  print(f"Prompt:\n{eval_prompt}\n")
  print(f"\nGenerated response:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(eval_prompt):]}")
  print('''\nGround truth:\ndef print_prime_factors(n): 
    for i in range(2, n + 1):
      while n % i == 0:
        print(i)
        n //= i
  print_prime_factors(n)''')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Prompt:
Write a function to output the prime factorization of 2023 in Java, Javascript, C, and C++


Generated response:
м

2023 is a prime number. Prime numbers are those numbers which are only divisible by 1 and itself. 2023 is the smallest prime number which is greater than 2000.

2023 = 1 * 2023

2023 = 1 * 2023

2023 = 1 * 2023

2023 = 1 * 2023

2023 = 1 * 2023

2023 = 1 * 2023

2023 = 1 * 2023

2023 = 1 * 2023

2023 = 1 * 2023

2023 = 1 * 2023

2023 = 1 * 2023

2023 = 1 * 2023

2023 = 1 * 2023

2023 = 1 * 20

Ground truth:
def print_prime_factors(n): 
    for i in range(2, n + 1):
      while n % i == 0:
        print(i)
        n //= i
  print_prime_factors(n)


## Evaluation 1

In [16]:
from tqdm import tqdm
predictions = []
ground_truths = []
count = 0
pbar = tqdm(total=min(0, TESTSIZE), desc="Processing test dataset")

model.eval()
with torch.no_grad():
  for test_prompt in test_dataset:
    if count == TESTSIZE:
      break
    eval_prompt = f"""<s>[INST] Write a {EXPERT} program to complete the following. {test_prompt['text']} [/INST] \\n </s>"""
    ground_truths.append(test_prompt['code'])
    input_ids = tokenizer(eval_prompt, return_tensors="pt", truncation=True).input_ids.cuda()
    outputs = model.generate(input_ids=input_ids, max_new_tokens=256, do_sample=True, top_p=0.9,temperature=0.5)
    prediction = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]
    predictions.append(prediction[len(eval_prompt):])
    count += 1
    pbar.update(1)

pbar.close()

Processing test dataset: 0it [00:00, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Processing test dataset: 1it [00:16, 16.44s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Processing test dataset: 2it [00:33, 16.51s/it]The attention mask and 

In [17]:
import pandas as pd

# Create a DataFrame with a single column named "value"
predictions_df = pd.DataFrame({"value": predictions})

# Path to the output JSONL file
predictions_output_file = f"{MODELTYPE}_predictions_{EXPERT}.jsonl"

# Write the DataFrame to the JSONL file
predictions_df.to_json(predictions_output_file, orient='records', lines=True)

In [18]:
predictions_df.head()

,value
0,2-25T05:03:48+05:30\n\n```\nimport java.util.*...
1,8-05T06:20:39.123Z\n\nThe above code is workin...
2,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1...
3,"3 , 4 , 5 , 6 , 7 , 8 , 9 , 10 , 11 , 12 , 13 ..."
4,0000000000000000000000000000000000000000000000...


In [19]:
references_df = pd.DataFrame({"value": ground_truths})
references_output_file = f"{MODELTYPE}_ground_truths_{EXPERT}.jsonl"
references_df.to_json(references_output_file, orient='records', lines=True)

## CodeBleu 1

In [21]:
from codebleu import calc_codebleu
results = []
for idx, reference in enumerate(ground_truths):
  prediction = predictions[idx]
  result = calc_codebleu([reference], [prediction], lang="java", weights=(0.25, 0.25, 0.25, 0.25), tokenizer=None)
  results.append(result)
print(result)
df = pd.DataFrame(results)
score_file = f'{MODELTYPE}_eval_report_{EXPERT}.csv'
df.to_csv(score_file)
# {
#   'codebleu': 0.5537,
#   'ngram_match_score': 0.1041,
#   'weighted_ngram_match_score': 0.1109,
#   'syntax_match_score': 1.0,
#   'dataflow_match_score': 1.0
# }

/opt/conda/lib/python3.10/site-packages/tree_sitter/__init__.py:36: FutureWarning: Language(path, name) is deprecated. Use Language(ptr, name) instead.
  warn("{} is deprecated. Use {} instead.".format(old, new), FutureWarning)


{'codebleu': 0.2511363246143541, 'ngram_match_score': 0.0021298668832422764, 'weighted_ngram_match_score': 0.0024154315741740004, 'syntax_match_score': 0.0, 'dataflow_match_score': 0.0}


In [22]:
df['codebleu'].mean()

0.18595100753087304

In [23]:
report_df = pd.concat([references_df, predictions_df, df], axis="columns")
report_file = f'{MODELTYPE}_eval_full_report_{EXPERT}.csv'
report_df.to_csv(report_file)

## Evaluation 2

In [24]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [25]:
from datasets import load_dataset
EXPERT = 'Javascript'
PARTITION = "Javascript-program-level" 
data = load_dataset(DATASET, PARTITION)
next(iter(data["test"]))

Generating validation split: 100%|██████████| 475/475 [00:00<00:00, 199369.00 examples/s]


{'text': 'Minimum sum possible by removing all occurrences of any array element | Function to find minimum sum after deletion ; Stores frequency of array elements ; Traverse the array ; Calculate sum ; Update frequency of the current element ; Stores the minimum sum required ; Traverse map ; Find the minimum sum obtained ; Return minimum sum ; Input array ; Size of array',
 'code': 'function minSum ( A , N ) { let mp = new Map ( ) ; let sum = 0 ; for ( let i = 0 ; i < N ; i ++ ) { sum += A [ i ] ; mp [ A [ i ] ] ++ ; if ( mp . has ( A [ i ] ) ) { mp . set ( A [ i ] , mp . get ( A [ i ] ) + 1 ) } else { mp . set ( A [ i ] , 1 ) } } let minSum = Number . MAX_SAFE_INTEGER ; for ( let it of mp ) { minSum = Math . min ( minSum , sum - ( it [ 0 ] * it [ 1 ] ) ) ; } return minSum ; } let arr = [ 4 , 5 , 6 , 6 ] ; let N = arr . length document . write ( minSum ( arr , N ) + " " ) ;'}

In [26]:
train_dataset = data["train"]
eval_dataset = data["validation"]
test_dataset = data["test"]
print(train_dataset)
print(eval_dataset)
print(test_dataset)

Dataset({
    features: ['text', 'code'],
    num_rows: 8590
})
Dataset({
    features: ['text', 'code'],
    num_rows: 475
})
Dataset({
    features: ['text', 'code'],
    num_rows: 886
})


In [27]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [28]:
from tqdm import tqdm
predictions = []
ground_truths = []
count = 0
pbar = tqdm(total=min(0, TESTSIZE), desc="Processing test dataset")

model.eval()
with torch.no_grad():
  for test_prompt in test_dataset:
    if count == TESTSIZE:
      break
    eval_prompt = f"""<s>[INST] Write a {EXPERT} program to complete the following. {test_prompt['text']} [/INST] \\n </s>"""
    ground_truths.append(test_prompt['code'])
    input_ids = tokenizer(eval_prompt, return_tensors="pt", truncation=True).input_ids.cuda()
    outputs = model.generate(input_ids=input_ids, max_new_tokens=256, do_sample=True, top_p=0.9,temperature=0.5)
    prediction = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]
    predictions.append(prediction[len(eval_prompt):])
    count += 1
    pbar.update(1)

pbar.close()

Processing test dataset: 0it [00:00, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Processing test dataset: 1it [00:16, 16.18s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Processing test dataset: 2it [00:32, 16.34s/it]The attention mask and 

In [29]:
import pandas as pd

# Create a DataFrame with a single column named "value"
predictions_df = pd.DataFrame({"value": predictions})

# Path to the output JSONL file
predictions_output_file = f"{MODELTYPE}_predictions_{EXPERT}.jsonl"

# Write the DataFrame to the JSONL file
predictions_df.to_json(predictions_output_file, orient='records', lines=True)

In [30]:
predictions_df.head()

,value
0,6789101112131415161718192021222324252627282930...
1,4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 2...
2,"исать программу на JavaScript, чтобы завершить..."
3,4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...
4,4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 2...


In [31]:
references_df = pd.DataFrame({"value": ground_truths})
references_output_file = f"{MODELTYPE}_ground_truths_{EXPERT}.jsonl"
references_df.to_json(references_output_file, orient='records', lines=True)

## CodeBleu 2

In [32]:
from codebleu import calc_codebleu
results = []
for idx, reference in enumerate(ground_truths):
  prediction = predictions[idx]
  result = calc_codebleu([reference], [prediction], lang="javascript", weights=(0.25, 0.25, 0.25, 0.25), tokenizer=None)
  results.append(result)
print(result)
df = pd.DataFrame(results)
score_file = f'{MODELTYPE}_eval_report_{EXPERT}.csv'
df.to_csv(score_file)
# {
#   'codebleu': 0.5537,
#   'ngram_match_score': 0.1041,
#   'weighted_ngram_match_score': 0.1109,
#   'syntax_match_score': 1.0,
#   'dataflow_match_score': 1.0
# }

/opt/conda/lib/python3.10/site-packages/tree_sitter/__init__.py:36: FutureWarning: Language(path, name) is deprecated. Use Language(ptr, name) instead.
  warn("{} is deprecated. Use {} instead.".format(old, new), FutureWarning)


{'codebleu': 0.25057657716707915, 'ngram_match_score': 0.0011930949277596827, 'weighted_ngram_match_score': 0.0011132137405568659, 'syntax_match_score': 0.0, 'dataflow_match_score': 0.0}


In [33]:
df['codebleu'].mean()

0.1908531860675776

In [34]:
report_df = pd.concat([references_df, predictions_df, df], axis="columns")
report_file = f'{MODELTYPE}_eval_full_report_{EXPERT}.csv'
report_df.to_csv(report_file)

## Evaluation 3

In [35]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [36]:
from datasets import load_dataset
EXPERT = 'C++'
PARTITION = "C++-program-level" 
data = load_dataset(DATASET, PARTITION)
next(iter(data["test"]))

Generating validation split: 100%|██████████| 492/492 [00:00<00:00, 202512.03 examples/s]


{'text': 'Minimum sum possible by removing all occurrences of any array element | C ++ program for the above approach ; Function to find minimum sum after deletion ; Stores frequency of array elements ; Traverse the array ; Calculate sum ; Update frequency of the current element ; Stores the minimum sum required ; Traverse map ; Find the minimum sum obtained ; Return minimum sum ; Driver code ; Input array ; Size of array',
 'code': '#include <bits/stdc++.h> NEW_LINE using namespace std ; int minSum ( int A [ ] , int N ) { map < int , int > mp ; int sum = 0 ; for ( int i = 0 ; i < N ; i ++ ) { sum += A [ i ] ; mp [ A [ i ] ] ++ ; } int minSum = INT_MAX ; for ( auto it : mp ) { minSum = min ( minSum , sum - ( it . first * it . second ) ) ; } return minSum ; } int main ( ) { int arr [ ] = { 4 , 5 , 6 , 6 } ; int N = sizeof ( arr ) / sizeof ( arr [ 0 ] ) ; cout << minSum ( arr , N ) << " STRNEWLINE " ; }'}

In [37]:
train_dataset = data["train"]
eval_dataset = data["validation"]
test_dataset = data["test"]
print(train_dataset)
print(eval_dataset)
print(test_dataset)

Dataset({
    features: ['text', 'code'],
    num_rows: 9797
})
Dataset({
    features: ['text', 'code'],
    num_rows: 492
})
Dataset({
    features: ['text', 'code'],
    num_rows: 909
})


In [38]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [39]:
from tqdm import tqdm
predictions = []
ground_truths = []
count = 0
pbar = tqdm(total=min(0, TESTSIZE), desc="Processing test dataset")

model.eval()
with torch.no_grad():
  for test_prompt in test_dataset:
    if count == TESTSIZE:
      break
    eval_prompt = f"""<s>[INST] Write a {EXPERT} program to complete the following. {test_prompt['text']} [/INST] \\n </s>"""
    ground_truths.append(test_prompt['code'])
    input_ids = tokenizer(eval_prompt, return_tensors="pt", truncation=True).input_ids.cuda()
    outputs = model.generate(input_ids=input_ids, max_new_tokens=256, do_sample=True, top_p=0.9,temperature=0.5)
    prediction = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]
    predictions.append(prediction[len(eval_prompt):])
    count += 1
    pbar.update(1)

pbar.close()

Processing test dataset: 0it [00:00, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Processing test dataset: 1it [00:16, 16.54s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Processing test dataset: 2it [00:33, 16.62s/it]The attention mask and 

In [40]:
import pandas as pd

# Create a DataFrame with a single column named "value"
predictions_df = pd.DataFrame({"value": predictions})

# Path to the output JSONL file
predictions_output_file = f"{MODELTYPE}_predictions_{EXPERT}.jsonl"

# Write the DataFrame to the JSONL file
predictions_df.to_json(predictions_output_file, orient='records', lines=True)

In [50]:
predictions_df.head()

,value
0,rogram to find the minimum sum possible by rem...
1,8-10 06:08:46 2020-08-10 06:08:46 [INST] Write...
2,4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 2...
3,4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 2...
4,017 2017 2017 2017 2017 2017 2017 2017 2017 20...


In [42]:
references_df = pd.DataFrame({"value": ground_truths})
references_output_file = f"{MODELTYPE}_ground_truths_{EXPERT}.jsonl"
references_df.to_json(references_output_file, orient='records', lines=True)

## CodeBleu 3

In [43]:
from codebleu import calc_codebleu
results = []
for idx, reference in enumerate(ground_truths):
  prediction = predictions[idx]
  result = calc_codebleu([reference], [prediction], lang="cpp", weights=(0.25, 0.25, 0.25, 0.25), tokenizer=None)
  results.append(result)
print(result)
df = pd.DataFrame(results)
score_file = f'{MODELTYPE}_eval_report_{EXPERT}.csv'
df.to_csv(score_file)
# {
#   'codebleu': 0.5537,
#   'ngram_match_score': 0.1041,
#   'weighted_ngram_match_score': 0.1109,
#   'syntax_match_score': 1.0,
#   'dataflow_match_score': 1.0
# }

/opt/conda/lib/python3.10/site-packages/tree_sitter/__init__.py:36: FutureWarning: Language(path, name) is deprecated. Use Language(ptr, name) instead.
  warn("{} is deprecated. Use {} instead.".format(old, new), FutureWarning)


{'codebleu': 0.10879641134863047, 'ngram_match_score': 0.0016919618686635266, 'weighted_ngram_match_score': 0.0016755017076765285, 'syntax_match_score': 0.0, 'dataflow_match_score': 0.4318181818181818}


In [44]:
df['codebleu'].mean()

0.21698470473013765

In [45]:
report_df = pd.concat([references_df, predictions_df, df], axis="columns")
report_file = f'{MODELTYPE}_eval_full_report_{EXPERT}.csv'
report_df.to_csv(report_file)